importing library

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
df = pd.read_csv("C:\Users\Sashi\Desktop\jupyter projects\final_dataset_123.csv",encoding='latin-1')
df.head(10)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2363989673.py, line 1)

In [16]:
df.drop('Crop',axis=1,inplace=True)

NameError: name 'df' is not defined

In [ ]:
df.drop('Season',axis=1,inplace=True)

In [ ]:
df.drop('Crop_Year',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['ANNUAL RAINFALL (mm)']

# ** Graph**

In [ ]:
plt.figure(figsize=(10,20))
sns.countplot(y=df['State_Name'])

In [ ]:
plt.figure(figsize=(10,20))
sns.countplot(y=df['Production'])

In [ ]:
df.head(2)

# **yield per district**

In [ ]:
state = (df['District_Name'].unique())

In [ ]:
yield_per_state = []
for i in state:
  yield_per_state.append(df[df['District_Name']==i]['Production'].sum())

In [ ]:
df['Production']

# **production per state**

In [ ]:
plt.figure(figsize=(10,20))
sns.barplot(y=yield_per_state,x=state,color='blue')

# **Train test split reaaranging columns**

In [ ]:
col =['State_Name','District_Name','temperature ©','humidity(%)','wind speed (m/s)','ANNUAL RAINFALL (mm)','soil type','Area','Production']
df = df[col]

In [ ]:
df

In [ ]:
x = df.drop('Production',axis=1)
y = df['Production']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
x_train

# **converting categorical to numerical and scalling the value**

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler # Changed oneHotEncoder to OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
ohe = OneHotEncoder(drop='first')
scaler = StandardScaler()

In [ ]:
x_train.head()

In [ ]:
preprocessing = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', ohe, [0, 1, 6]),
        ('Standerization', scaler, [2, 3, 4, 5, 7])
    ],
    remainder='passthrough'
)

In [ ]:
preprocessing

In [ ]:
x_train_dummy = preprocessing.fit_transform(x_train) # Changed preprocesser to preprocessing
x_test_dummy = preprocessing.transform(x_test)

In [ ]:
x_train_dummy

# **Training model**

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error

models = {
    'Lr': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'KNr': KNeighborsRegressor(),
    'dtr': DecisionTreeRegressor()
}


In [ ]:
for name, model in models.items():
    model.fit(x_train_dummy, y_train)
    y_pred = model.predict(x_test_dummy)

    print(f"{name}MSE:{mean_squared_error(y_test, y_pred)} score {r2_score(y_test, y_pred)}")

# **Select model**


In [ ]:
Dtr = DecisionTreeRegressor()
Dtr.fit(x_train_dummy, y_train)
Dtr.predict(x_test_dummy)

# **predictive system**

In [ ]:
def prediction(state_Name,District_Name,temperature ,humidity,wind_speed,RAINFALL,soil_type,Area):
  features =np.array([state_Name,District_Name,temperature,humidity,wind_speed,RAINFALL,soil_type,Area])
  features = pd.DataFrame([features], columns=['State_Name','District_Name','temperature ©','humidity(%)','wind speed (m/s)','ANNUAL RAINFALL (mm)','soil type','Area'])

  transformed_feature = preprocessing.transform(features)
  predicted_value = Dtr.predict(transformed_feature).reshape(1,-1)
  return predicted_value[0][0]

In [ ]:
state_Name ='Rajasthan'
District_Name ='TONK'
temperature =	26.10
humidity =41.19
wind_speed =3.55
RAINFALL =703.0
soil_type ='alluvial'
Area=255

result = prediction(state_Name,District_Name,temperature,humidity,wind_speed,RAINFALL,soil_type,Area)


In [ ]:
result

# **Pickel**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle
pickle.dump(Dtr, open('Dtr.pkl', 'wb'))
pickle.dump(preprocessing, open('preprocessor.pkl', 'wb'))